In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
import skimage.morphology
import skimage.segmentation

import glob
import os
from tqdm import tqdm

import evaluation

/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Scoring function

In [3]:
def compute_scores(gt_path, predictions_path, submission_id, extension=".png"):
    results = pd.DataFrame(columns=["Image", "Threshold", "F1", "Jaccard", "TP", "FP", "FN", "Official_Score", "Precision", "Recall"])
    false_negatives = pd.DataFrame(columns=["False_Negative", "Area"])
    splits_merges = pd.DataFrame(columns=["Image_Name", "Merges", "Splits"])

    for image_name in y_true_identifiers:
        # Load ground truth data
        img_filename = os.path.join(gt_path, image_name + ".png")
        ground_truth = skimage.io.imread(img_filename)
        
        if len(ground_truth.shape) == 3:
            ground_truth = ground_truth[:,:,0]

        # Transform to label matrix
        ground_truth = skimage.morphology.label(ground_truth)

        # Load predictions
        pred_filename = os.path.join(predictions_path, str(submission_id) + "-" + image_name + extension)
        if os.path.isfile(pred_filename):
            prediction = skimage.io.imread(pred_filename)
        else:
            prediction = np.zeros_like(ground_truth)

        # Relabel objects
        ground_truth = skimage.segmentation.relabel_sequential(ground_truth)[0] 
        prediction = skimage.segmentation.relabel_sequential(prediction)[0] 

        # Compute evaluation metrics
        results = evaluation.compute_af1_results(
            ground_truth, 
            prediction, 
            results, 
            image_name
        )

        false_negatives = evaluation.get_false_negatives(
            ground_truth, 
            prediction, 
            false_negatives, 
            image_name
        )

        splits_merges = evaluation.get_splits_and_merges(
            ground_truth, 
            prediction, 
            splits_merges, 
            image_name
        )
        
    return results, false_negatives, splits_merges

# Load metadata

In [4]:
submissions = pd.read_csv("/storage/data/DSB2018/metadata/submissions.csv")

submissions = submissions[submissions["score"] > 0]

submissions["max_score"] = submissions.groupby(["team"])["score"].transform(max)

submissions["best_submission"] = submissions["max_score"] == submissions["score"]

submissions = submissions[submissions["best_submission"]]

In [5]:
y_true_pathnames = glob.glob("/storage/data/DSB2018/phase-2-gt/*.png")

y_true_identifiers = [os.path.splitext(os.path.basename(pathname))[0] for pathname in y_true_pathnames]

# Compute scores for all submissions

In [6]:
accuracy = pd.DataFrame(columns=["Image", "Threshold", "F1", "Jaccard", "TP", "FP", "FN", "Official_Score", "SubmissionID"])
false_negatives = pd.DataFrame(columns=["False_Negative", "Area", "SubmissionID"])
splits_merges = pd.DataFrame(columns=["Image_Name", "Merges", "Splits", "SubmissionID"])

gt_path = "/storage/data/DSB2018/phase-2-gt/"
predictions_path = "/storage/data/DSB2018/phase-2-predictions/"

for idx in tqdm( submissions["team"].unique() ):
    record = submissions[submissions["team"] == idx]
    
    # Take the first row. Repeated records have the same score
    submission_id = record.iloc[0].id
    accu, falsen, spl_mer = compute_scores(gt_path, predictions_path, submission_id)
    accu["SubmissionID"] = submission_id
    falsen["SubmissionID"] = submission_id
    spl_mer["SubmissionID"] = submission_id
    
    # Append results
    accuracy = pd.concat([accuracy, accu])
    false_negatives = pd.concat([false_negatives, falsen])
    splits_merges = pd.concat([splits_merges, spl_mer])

  0%|          | 0/706 [00:00<?, ?it/s]/home/jccaicedo/dsb2018-analysis/evaluation.py:105: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  results = pd.concat([results, pd.DataFrame(data=data.T, columns=["Area", "False_Negative"])])
/home/jccaicedo/dsb2018-analysis/evaluation.py:68: RuntimeWarning: Mean of empty slice.
  jaccard = np.max(IOU, axis=0).mean()
/home/jccaicedo/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

T

In [7]:
accuracy.to_csv("/storage/data/DSB2018/results/accuracy.csv", index=False)
false_negatives.to_csv("/storage/data/DSB2018/results/false_negatives.csv", index=False)
splits_merges.to_csv("/storage/data/DSB2018/results/splits_merges.csv", index=False)

# CellProfiler baseline

In [8]:
gt_path = "/storage/data/DSB2018/phase-2-gt/"
predictions_path = "/storage/data/DSB2018/CP_segmentations/all/"

submission_id = "cellprofiler"
accu, falsen, spl_mer = compute_scores(gt_path, predictions_path, submission_id, extension=".tiff")
accu["SubmissionID"] = submission_id
falsen["SubmissionID"] = submission_id
spl_mer["SubmissionID"] = submission_id

accu.to_csv("/storage/data/DSB2018/results/cp_accuracy.csv", index=False)
falsen.to_csv("/storage/data/DSB2018/results/cp_false_negatives.csv", index=False)
spl_mer.to_csv("/storage/data/DSB2018/results/cp_splits_merges.csv", index=False)

/home/jccaicedo/dsb2018-analysis/evaluation.py:105: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  results = pd.concat([results, pd.DataFrame(data=data.T, columns=["Area", "False_Negative"])])
